In [ ]:

import sys
 
# adding Folder_2 to the system path
sys.path.insert(0, r'F:\f1analytics\scripts')


import requests as rq

from converter import convert_ntt
import requests as rq
from sqlalchemy import create_engine

from multiprocessing import Process
from multiprocessing import Manager
import time

engine=create_engine('')
conn=engine.connect()
def getData(ttList):
    while True:
        ttList.append(rq.get(r'https://f1.tfeed.net/tt.js').text)
def toDB(ttList):
    text=ttList.pop(0)
    tlmy,wthr,race=convert_ntt(text)
    fullDf=tlmy.merge(wthr,on='timestamp').merge(race,on='timestamp')
    fullDf['session_name']='2024_australia'
    fullDf.to_sql('testing',conn,if_exists='append')


if __name__=='__main__':
    mgr = Manager()
    ttList=mgr.list()

    p1=Process(target=getData,args=(ttList,))
    p2=Process(target=toDB,args=(ttList,))
    p1.start()
    time.sleep(5)
    p2.start()
 